In [25]:
import pandas as pd
import numpy as np
import re

from tqdm import tqdm

In [123]:
cosing_ori = pd.read_csv('../Database/COSING_Ingredients-Fragrance Inventory_v2.csv')

In [124]:
cosing_ori.head()

,COSING Ref No,INCI name,INN name,Ph. Eur. Name,CAS No,EC No,Chem/IUPAC Name / Description,Restriction,Function,Update Date
0,94753,DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORM...,NaN,NaN,"2040469-40-5, 2422121-34-2",,Disodium Tetramethylhexadecenylcysteine Formyl...,NaN,SKIN PROTECTING,16/06/2020
1,94896,(LIQUIDAMBAR STYRACIFLUA/TRIBULUS TERRESTRIS)...,NaN,NaN,,,(Liquidambar Styraciflua/Tribulus Terrestris)...,NaN,SKIN CONDITIONING,01/12/2017
2,95645,ACRYLATES/VA/VINYL NEODECANOATE COPOLYMER,NaN,NaN,99728-55-9,,Acrylates/VA/Vinyl Neodecanoate Copolymer is ...,NaN,PLASTICISER,14/02/2018
3,96229,ASTROCARYUM VULGARE SEED BUTTER,NaN,NaN,2187449-53-0,,Astrocaryum Vulgare Seed Butter is the fat obt...,NaN,"SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIENT",06/04/2018
4,89177,BARLEY SH-POLYPEPTIDE-17,NaN,NaN,,,Barley sh-Polypeptide-17 is a single chain rec...,NaN,"HAIR CONDITIONING, SKIN CONDITIONING",09/03/2018


In [125]:
cosing_ori.drop(columns=['EC No', 'Update Date', 'CAS No'], inplace=True)

# lower > upper case for INN name and Ph.Eur Name
cosing_ori['INN name'] = cosing_ori['INN name'].apply(lambda x: str(x).upper() if not pd.isna(x) else x)
cosing_ori['Ph. Eur. Name'] = cosing_ori['Ph. Eur. Name'].apply(lambda x: str(x).upper() if not pd.isna(x) else x)

In [126]:
# cosing_ori[~cosing_ori['INN name'].isna()]

In [127]:
# cosing_ori[['INCI name', 'INN name', 'Ph. Eur. Name']]

In [114]:
def clean_string_name(string):
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex1 = re.compile('\[.*]')
    regex2 = re.compile('\[.*}')
    text = re.sub(pattern, "", string)
    text = re.sub(regex1, "", text)
    text = re.sub(regex2, "", text)
    text = text.strip('\x0c')
    text = text.strip(' ')
    return text

In [128]:
clean_string_name(coslist[0])

'DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYLPROLINATE'

In [129]:
def append_names(x):
    inn = clean_string_name(str(x[1]))
    eur = clean_string_name(str(x[2]))
    inci = clean_string_name(str(x[0]))
    if pd.isna(x[1]) and pd.isna(x[2]):
        x[0] = inci
    
    elif (not pd.isna(x[1])) and inn != x[0] and pd.isna(x[2]):
        x[0] = inci + '/' + inn
    
    elif (not pd.isna(x[1])) and inn == x[0] and pd.isna(x[2]):
        x[0] = inci
        
    elif pd.isna(x[1]) and eur != x[0] and (not pd.isna(x[2])):
        x[0] = inci + '/' + eur
    
    elif pd.isna(x[1]) and eur == x[0] and (not pd.isna(x[2])):
        x[0] = inci
    
    elif (not pd.isna(x[1])) and (not pd.isna(x[2])) and inn == x[0]:
        x[0] = inci + '/' + eur
                                  
    elif (not pd.isna(x[1])) and (not pd.isna(x[2])) and eur == x[0]:
        x[0] = inci + '/' + inn
                                  
    else:   
        x[0] = inci + '/' + inn + '/' + eur
        
    return x[0]

In [130]:
cosing_ori['INCI name'] = cosing_ori[['INCI name', 'INN name', 'Ph. Eur. Name']].apply(append_names, axis=1)

In [131]:
cosing_ori.head()

,COSING Ref No,INCI name,INN name,Ph. Eur. Name,Chem/IUPAC Name / Description,Restriction,Function
0,94753,DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYL...,NaN,NaN,Disodium Tetramethylhexadecenylcysteine Formyl...,NaN,SKIN PROTECTING
1,94896,(LIQUIDAMBAR STYRACIFLUA/TRIBULUS TERRESTRIS) ...,NaN,NaN,(Liquidambar Styraciflua/Tribulus Terrestris)...,NaN,SKIN CONDITIONING
2,95645,ACRYLATES/VA/VINYL NEODECANOATE COPOLYMER,NaN,NaN,Acrylates/VA/Vinyl Neodecanoate Copolymer is ...,NaN,PLASTICISER
3,96229,ASTROCARYUM VULGARE SEED BUTTER,NaN,NaN,Astrocaryum Vulgare Seed Butter is the fat obt...,NaN,"SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIENT"
4,89177,BARLEY SH-POLYPEPTIDE-17,NaN,NaN,Barley sh-Polypeptide-17 is a single chain rec...,NaN,"HAIR CONDITIONING, SKIN CONDITIONING"


In [132]:
cosing_ori[~cosing_ori['INN name'].isna()][['INCI name', 'INN name', 'Ph. Eur. Name']].tail(60)
cosing_ori.drop(columns=['INN name', 'Ph. Eur. Name'], inplace=True)

In [133]:
cosing_ori

,COSING Ref No,INCI name,Chem/IUPAC Name / Description,Restriction,Function
0,94753,DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYL...,Disodium Tetramethylhexadecenylcysteine Formyl...,NaN,SKIN PROTECTING
1,94896,(LIQUIDAMBAR STYRACIFLUA/TRIBULUS TERRESTRIS) ...,(Liquidambar Styraciflua/Tribulus Terrestris)...,NaN,SKIN CONDITIONING
2,95645,ACRYLATES/VA/VINYL NEODECANOATE COPOLYMER,Acrylates/VA/Vinyl Neodecanoate Copolymer is ...,NaN,PLASTICISER
3,96229,ASTROCARYUM VULGARE SEED BUTTER,Astrocaryum Vulgare Seed Butter is the fat obt...,NaN,"SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIENT"
4,89177,BARLEY SH-POLYPEPTIDE-17,Barley sh-Polypeptide-17 is a single chain rec...,NaN,"HAIR CONDITIONING, SKIN CONDITIONING"
...,...,...,...,...,...
28570,91114,ACRYLOYL DIMETHYL TAURATE/MELAMINE/PEG-6 METH...,Acryloyl Dimethyl Taurate/Melamine/PEG-6 Metha...,NaN,BINDING
28571,90707,BACILLUS/(ASPARAGUS COCHINCHINENSIS/LYCIUM CH...,Bacillus/(Asparagus Cochinchinensis/Lycium Chi...,NaN,SKIN CONDITIONING
28572,90452,LACTOBACILLUS/HYDROLYZED FERMENT EXTRACT FIL...,: This is a filtrate of the extract of the pro...,NaN,SKIN CONDITIONING
28573,90014,LACTOBACILLUS/RHODOPSEUDOMONAS/SACCHAROMYCES/...,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"


In [134]:
coslist = cosing_ori['INCI name'].to_list()
coslist

['DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYLPROLINATE',
 '(LIQUIDAMBAR STYRACIFLUA/TRIBULUS TERRESTRIS) FRUIT/PAEONIA LACTIFLORA ROOT EXTRACT',
 'ACRYLATES/VA/VINYL NEODECANOATE COPOLYMER',
 'ASTROCARYUM VULGARE SEED BUTTER',
 'BARLEY SH-POLYPEPTIDE-17',
 'DAUCUS CAROTA SATIVA LEAF EXTRACT',
 'GOSSYPIUM HIRSUTUM SEED EXTRACT',
 'HYDROLYZED (CITRUS AURANTIUM AMARA /FORSYTHIA VIRIDISSIMA/GARDENIA FLORIDA) FRUIT/MENTHA ARVENSIS LEAF/(ANGELICA ACUTILOBA/COPTIS JAPONICA/GLYCYRRHIZA GLABRA /PAEONIA LACTIFLORA/PLATYCODON GRANDIFLORUS/REHMANNIA GLUTINOSA/SCUTELLARIA BAICALENSIS ROOT) EXTRACT',
 'HYDROLYZED BEE LARVA EXTRACT',
 'HYDROLYZED FORSYTHIA VIRIDISSIMA FRUIT/(CIMICIFUGA DAHURICA/GLYCYRRHIZA GLABRA /PAEONIA LACTIFLORA/PLATYCODON GRANDIFLORUS/PUERARIA LOBATA) ROOT EXTRACT',
 'ICTASOL',
 'ISOOCTANOYL TETRAPEPTIDE-25',
 'LACTOBACILLUS/GERMINATED SOYBEAN FERMENT FILTRATE',
 'METHOXYETHANOL',
 'MYRISTOYL GLYCINE/HISTIDINE/LYSINE POLYPEPTIDE',
 'NONAPEPTIDE-11',
 'PENTACLETHRA MACROPHYLLA

In [135]:
new_df = pd.DataFrame()
res = cosing_ori.copy()
for idx, row in tqdm(res.iterrows()):
    names = row['INCI name'].split('/')
    if len(names) == 1:
        res.loc[idx, 'INCI name'] = res.loc[idx, 'INCI name']

    elif len(names) > 1:
#         print(names)
        new_row = row
        for name in names:
            new_row['INCI name'] = name
            new_df = new_df.append(row)
        res.drop(idx, axis=0, inplace=True)
res = res.append(new_df)
res = res.reset_index(drop=True)   

28575it [02:38, 180.51it/s]


In [136]:
res

,COSING Ref No,INCI name,Chem/IUPAC Name / Description,Restriction,Function
0,94753.0,DISODIUM TETRAMETHYLHEXADECENYLCYSTEINE FORMYL...,Disodium Tetramethylhexadecenylcysteine Formyl...,NaN,SKIN PROTECTING
1,96229.0,ASTROCARYUM VULGARE SEED BUTTER,Astrocaryum Vulgare Seed Butter is the fat obt...,NaN,"SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIENT"
2,89177.0,BARLEY SH-POLYPEPTIDE-17,Barley sh-Polypeptide-17 is a single chain rec...,NaN,"HAIR CONDITIONING, SKIN CONDITIONING"
3,98580.0,DAUCUS CAROTA SATIVA LEAF EXTRACT,Daucus Carota Sativa (Carrot) Leaf Extract is ...,NaN,SKIN CONDITIONING - MISCELLANEOUS
4,89078.0,GOSSYPIUM HIRSUTUM SEED EXTRACT,Gossypium Hirsutum Seed Extract is the extrac...,NaN,SKIN CONDITIONING
...,...,...,...,...,...
37304,90014.0,PHELLODENDRON AMURENSE BARK,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37305,90014.0,LONICERA JAPONICA FLOWER,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37306,90014.0,CHAENOMELES SINENSIS FRUIT,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"
37307,90014.0,CAMELLIA SINENSIS LEAF,This is a filtrate of the product obtained by ...,NaN,"HUMECTANT, SKIN CONDITIONING, SKIN PROTECTING"


In [137]:
res.to_csv('ingredient_cosing_37309.csv')